In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f6d4504521e3ffb40cdc2e4155616e0910d6cc36a6402f85da74f2eac4e3b2d7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import json
def save_json(dicc,file):
    with open(file,'w') as f:
        f.write(json.dumps(dicc))

In [3]:
def read_json(file):
    with open(file,'r') as f:
        data = json.load(f)
    return data

In [ ]:
!pip install FlightRadarAPI

  Using cached flightradarapi-1.3.27-py3-none-any.whl (14 kB)
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.0 MB)


In [ ]:
from FlightRadar24 import FlightRadar24API
import re
fr_api = FlightRadar24API()

## Obtain airports from cities we are interested in\

In [ ]:
airports = fr_api.get_airports()

In [ ]:
# List of cities
cities = ['Amsterdam','Athens','Barcelona','Berlin','Budapest','Lisbon','London','Paris','Rome','Vienna']

In [ ]:
import re
regex = r'\((.*?)\)'
airports_in_city = []

for ap in airports:
    for city in cities:
        if city in str(ap):
            acronym = re.findall(regex,str(ap))[0]
            airports_in_city.append(ap)
            break

In [ ]:
def parseig_aeroports(lista_aeropuertos):
    airports_info = {}
    regex = r'\((.*?)\) .*? Latitude: ([-+]?\d+\.\d+) - Longitude: ([-+]?\d+\.\d+)'

    for aeropuerto in lista_aeropuertos:
        matches = re.findall(regex, str(aeropuerto))
        if matches:
            siglas = matches[0][0]
            latitude = float(matches[0][1])
            longitude = float(matches[0][2])
            for city in cities:
                if city in aeropuerto:
                    airp_city = city
            airports_info[siglas] = {'Latitude': latitude, 'Longitude': longitude,'City':airp_city}

    return airports_info

In [ ]:
airports_info = parseig_aeroports(airports_in_city)

We store information we have by the moment because we had some issues with API requests.

In [ ]:
save_json(airports_info,'airports_info.json')

## Get arrivals from airports

In [ ]:
airports_info = read_json('airports_info.json')

In [ ]:
def get_100_arrivals(acronym):
    aeroport = fr_api.get_airport_details(acronym)
    vols = aeroport['airport']['pluginData']['schedule']['arrivals']['data']
    return vols

In [ ]:
for airp in airports_info.keys():
    airports_info[airp]['arrivals'] = get_100_arrivals(airp)

In [ ]:
save_json(airports_info,'airport_info_arrivals.json')

## Save into DataLake

In [4]:
import re

In [5]:
airports_info = read_json('airport_info_arrivals.json')

In [6]:
from pyspark.sql import SparkSession

# Init session in Spark
spark = SparkSession.builder \
    .appName("JSON a Parquet con Spark SQL") \
    .getOrCreate()

# Read json and create a temporal view
df = spark.read.json("airport_info_arrivals.json")
df.createOrReplaceTempView("airport_info_arrivals")

# Use Spark SQL to select the data
selected_data = spark.sql("SELECT * FROM airport_info_arrivals")


In [7]:
from pyspark.sql import SparkSession

# Init session in Spark
spark = SparkSession.builder \
    .appName("JSON a Parquet con Spark SQL") \
    .getOrCreate()

# Read json and create a temporal view
df = spark.read.json("airport_info_arrivals.json")
df.createOrReplaceTempView("airport_info_arrivals")

# Use Spark SQL to select the data
selected_data = spark.sql("SELECT * FROM airport_info_arrivals")

# Save data as a parquet file
selected_data.write.parquet("datalake/airport_info_arrivals.parquet")

# Stop Spark Session
spark.stop()

We realized that out json didn't have a great format to manage the data as we want. So we need to change it to have the following structure to begin:

| Airport acronym | Longitude | Latitude | City  | arrivals |
| --------------- | --------- | -------- | ----  | -------- |
| EHAM | 52.308609 | 4.763889 | Amsterdam  | [flight 3473cc55] |

To get this new structure we'll use map reduce, as we have seen in class. Then we'll built another parquet file and it will be the definitive of our datalake.

In [8]:
# Map function: transform input into intermediate key-value output
def map_function(input_data):
    intermediate = []
    for key, value in input_data.items():
        # The key is the airport acronym, and the value is the corresponding information
        intermediate.append((key, value))
    return intermediate

# Reduce function: process intermediate values for each key and emit final result
# In this case, each key is unique, so we just pass the data through
def reduce_function(intermediate_data):
    result = []
    for key, values in intermediate_data:
        # Transform the value to the desired structure (in this case, it's already in the desired structure)
        result.append((key, values['Longitude'], values['Latitude'], values['City'], values['arrivals']))
    return result

# Simulate the MapReduce process
intermediate = map_function(airports_info)
final_result = reduce_function(intermediate)

# Display the result
for item in final_result:
    print(item)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Convert json info into columns

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, DoubleType

In [10]:
flight_schema = schema = StructType([
    StructField("identification", StructType([
        StructField("number", StructType([
            StructField("default", StringType(), True)
        ]), True)
    ]), True),
    StructField("time", StructType([
        StructField("estimated", StructType([
            StructField("arrival", StringType(), True),
            StructField("departure", StringType(), True)
        ]), True),
        StructField("real", StructType([
            StructField("arrival", StringType(), True),
            StructField("departure", StringType(), True)
        ]), True),
        StructField("scheduled", StructType([
            StructField("arrival", StringType(), True),
            StructField("departure", StringType(), True)
        ]), True)
    ]), True),
    StructField("owner", StructType([
            StructField("name", StringType(), True),
            StructField("logo", StringType(), True)
        ]), True),
    StructField("airport", StructType([
        StructField("destination", StructType([
            StructField("info", StructType([
                StructField("gate", StringType(), True),
                StructField("terminal", StringType(), True),
                StructField("baggage", StringType(), True)
            ]), True)
        ]), True),
        StructField("real", StringType(), True),
        StructField("origin", StructType([
            StructField("name", StringType(), True),
            StructField("code", StructType([
                StructField("iata", StringType(), True)
            ]), True),
            StructField("position", StructType([
                StructField("region", StructType([
                    StructField("city", StringType(), True)
                ]), True),
                StructField("latitude", StringType(), True),
                StructField("longitude", StringType(), True)
            ]), True),
            StructField("info", StructType([
                StructField("gate", StringType(), True),
                StructField("terminal", StringType(), True)
            ]), True)
        ]), True)
    ]), True),
    StructField("status", StructType([
        StructField("live", StringType(), True)
    ]), True)
])

In [11]:
# Define the DataFrame schema according to the desired structure
schema = StructType([
    StructField("airport_acronym", StringType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("city", StringType(), True),
    StructField("arrivals", ArrayType(StructType([
    StructField("flight", flight_schema
    , True)
])), True)
])

# Start a Spark session
spark = SparkSession.builder \
    .appName("Load and Save Transformed Data") \
    .getOrCreate()

# Create a DataFrame from the transformed data and the defined schema
df = spark.createDataFrame(data=final_result, schema=schema)

# Show the DataFrame to verify its content
df.show()
df.printSchema()

+---------------+----------+---------+---------+--------------------+
|airport_acronym| longitude| latitude|     city|            arrivals|
+---------------+----------+---------+---------+--------------------+
|           EHAM|  4.763889|52.308609|Amsterdam|[{{{{OS373}}, {{N...|
|           KAHN|-83.324722| 33.95195|   Athens|[{{{{NULL}}, {{17...|
|           LGAV|  23.94446|37.936352|   Athens|[{{{{A3247}}, {{N...|
|           KUNI|  -82.2314|39.210999|   Athens|                  []|
|           KMMI|  -84.5625|35.397221|   Athens|                  []|
|           LEBL|  2.078463| 41.29707|Barcelona|[{{{{VY2219}}, {{...|
|           SVBC|-64.689102| 10.10713|Barcelona|[{{{{CM328}}, {{1...|
|           EDDB| 13.503722|52.362877|   Berlin|[{{{{BA994}}, {{1...|
|           LHBS| 18.980589|47.451065| Budapest|                  []|
|           LHBP|  19.25559|47.436932| Budapest|[{{{{W62222}}, {{...|
|           FAEL|  27.82593| -33.0355|   London|[{{{{4Z917}}, {{1...|
|           LPPT|  -

In [13]:
df_ex = df.withColumn("arrival", F.explode("arrivals")).drop("arrivals")

In [14]:
df_formatted = df_ex \
    .withColumn("flight_number_default", F.col("arrival.flight.identification.number.default")) \
    .withColumn("estimated_arrival_time", F.col("arrival.flight.time.estimated.arrival")) \
    .withColumn("estimated_departure_time", F.col("arrival.flight.time.estimated.departure")) \
    .withColumn("real_arrival_time", F.col("arrival.flight.time.real.arrival")) \
    .withColumn("real_departure_time", F.col("arrival.flight.time.real.departure")) \
    .withColumn("scheduled_arrival_time", F.col("arrival.flight.time.scheduled.arrival")) \
    .withColumn("scheduled_departure_time", F.col("arrival.flight.time.scheduled.departure")) \
    .withColumn("owner_name", F.col("arrival.flight.owner.name")) \
    .withColumn("owner_logo", F.col("arrival.flight.owner.logo")) \
    .withColumn("destination_gate", F.col("arrival.flight.airport.destination.info.gate")) \
    .withColumn("destination_terminal", F.col("arrival.flight.airport.destination.info.terminal")) \
    .withColumn("destination_baggage", F.col("arrival.flight.airport.destination.info.baggage")) \
    .withColumn("origin_airport", F.col("arrival.flight.airport.origin.code.iata")) \
    .withColumn("origin_city", F.col("arrival.flight.airport.origin.position.region.city")) \
    .withColumn("origin_latitude", F.col("arrival.flight.airport.origin.position.latitude")) \
    .withColumn("origin_longitude", F.col("arrival.flight.airport.origin.position.longitude")) \
    .withColumn("origin_gate", F.col("arrival.flight.airport.origin.info.gate")) \
    .withColumn("origin_terminal", F.col("arrival.flight.airport.origin.info.terminal")) \
    .withColumn("flight_status_live", F.col("arrival.flight.status.live")).drop('arrival')

In [15]:
df_formatted.show()

+---------------+---------+---------+---------+---------------------+----------------------+------------------------+-----------------+-------------------+----------------------+------------------------+--------------------+--------------------+----------------+--------------------+-------------------+--------------+-----------+---------------+----------------+-----------+---------------+------------------+
|airport_acronym|longitude| latitude|     city|flight_number_default|estimated_arrival_time|estimated_departure_time|real_arrival_time|real_departure_time|scheduled_arrival_time|scheduled_departure_time|          owner_name|          owner_logo|destination_gate|destination_terminal|destination_baggage|origin_airport|origin_city|origin_latitude|origin_longitude|origin_gate|origin_terminal|flight_status_live|
+---------------+---------+---------+---------+---------------------+----------------------+------------------------+-----------------+-------------------+----------------------+

In [16]:
# Save the DataFrame as a Parquet file
df_formatted.write.parquet("flights_data.parquet")

# Stop the Spark session
spark.stop()